# 본문 스크래핑

In [1]:
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm
import random
import urllib.request
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException, TimeoutException
from selenium import webdriver
from selenium.webdriver.common.by import By
# import chromedriver_auto


# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)


In [2]:
from user_agent import generate_user_agent, generate_navigator

print(generate_user_agent(device_type='desktop'))
print(generate_user_agent(os='win', device_type='desktop'))

navigator = generate_navigator()

random_user = generate_user_agent(os='win', device_type='desktop')
print(random_user)

Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0
Mozilla/5.0 (Windows NT 6.1; WOW64; rv:45.0) Gecko/20100101 Firefox/45.0
Mozilla/5.0 (Windows NT 6.3; Win64; x64; rv:51.0) Gecko/20100101 Firefox/51.0


In [4]:
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument(random_user)
driver = webdriver.Chrome(executable_path ='./chromedriver_win32/chromedriver.exe',options = chrome_options)

driver.maximize_window()
driver.get("https://www.naver.com/")
time.sleep(2)


C:\Users\user\AppData\Local\Temp\ipykernel_14308\251865567.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path ='./chromedriver_win32/chromedriver.exe',options = chrome_options)


In [5]:
url_df = pd.read_csv("./나무문화/2020/나무문화_링크_2020_9월.csv", encoding='UTF-8')
print(url_df.shape, url_df.columns)
url_df


(11904, 3) Index(['url', 'title', 'date'], dtype='object')


,url,title,date
0,https://blog.naver.com/jinidag/222104033228,추석연휴 가볼만한곳 한국민속촌 야외 당일치기 나들이,2020.09.30.
1,https://blog.naver.com/taya2765/222104032204,"동래 가볼만한 곳, 복천박물관",2020.09.30.
2,https://blog.naver.com/yeonjae1213/222104028784,배꼽 빠질 뻔했다,2020.09.30.
3,https://blog.naver.com/sunnybridge_/222104026647,"내가 사랑했던 마을,탬워스(Tamworth)",2020.09.30.
4,https://blog.naver.com/lby56/222104022581,언택트 한가위… ‘꼰대’ 삼촌이 10대 조카에게 보내는 편지,2020.09.30.
...,...,...,...
11899,https://blog.naver.com/chookk7/222076688184,“먹튀자본 MBK 김병주의 땅투기로 안산이 망가진다”,2020.09.01.
11900,https://blog.naver.com/bugman1303/222076686202,머리가 좋은 동물들,2020.09.01.
11901,https://blog.naver.com/sherry8279/222076052955,"<도시는 무엇으로 사는 가, 유현준> 도시의 다채로운 생각과...",2020.09.01.
11902,https://blog.naver.com/butterfly0724/222071335341,[경기도 미술관 카페] 호이안,2020.09.01.


In [7]:
blog_links = url_df['url'].to_list()
print(len(blog_links))


11904


In [8]:
#### 삭제/비공개 링크 인덱스도 매번 바꿔줄 것!!! ###
contents = []
old_blogs = []
hide_link = []

for i in tqdm(blog_links):
    driver.get(i)
    driver.implicitly_wait(10)
    try:
        driver.switch_to.frame("mainFrame")
    except TimeoutException as e:
        print('시간 초과 오류 : ', e)
        contents.append('수집불가능')
    except UnexpectedAlertPresentException as e:
    ### 이 오류 해결함. 에러 코드 파일에서 직접 임포트하면 된다!
        print(e)
                                                                            # 여기도 바꿔야 함
        print('{}번째 게시글은 비공개로 전환되었거나 게시판이 바뀜.'.format(blog_links.index(i)))
        hide_link.append(i)
        time.sleep(1.5)
        driver.switch_to.frame('mainFrame')
    try:
        a = driver.find_element(By.CSS_SELECTOR,'div.se-main-container').text
        contents.append(a)
    # NoSuchElement 오류시 예외처리(구버전 블로그에 적용)
    except NoSuchElementException:
        a = driver.find_element(By.CSS_SELECTOR,'div#content-area').text
        contents.append(a)
        old_blogs.append(i)


  0%|          | 0/11904 [00:00<?, ?it/s]

In [36]:
print(len(contents))
print(len(old_blogs))
print(len(hide_link))


1000
28
0


In [37]:
df17 = pd.DataFrame({'url':blog_links17, 'title':url_df['title'][16000:17000], 'date':url_df['date'][16000:17000], '본문':contents})
df17

,url,title,date,본문
16000,https://blog.naver.com/skstjs4148/222894238343,대구 가볼만한곳 / 팔공산 파계사 / 살짝 만나본 가을산 단풍...,2022.10.12.,대구 가볼만한곳\n<팔공산 파계사>\n연락처 : 053-984-4550\nNext ...
16001,https://blog.naver.com/kskim2019/222898016849,"신간책 소개 ""하얀밤의 고양이""",2022.10.12.,나무와열매작은도서관 책 소개\n10월에 입고된 「교보문학나눔」 책을 소개합니다!\n...
16002,https://blog.naver.com/bluewablack/222898014538,서울억새축제 2022.10.15 ~ 10.21,2022.10.12.,서울억새축제\n3년만에 돌아온 서울억새축제\n일정 미리 챙기세요\n50m\n© NA...
16003,https://blog.naver.com/banabana2022/222898013604,[전북 전주]전주전동성당 미사시간/전주홍삼파는화가 육전...,2022.10.12.,전동성당/홍삼파는화가/오목대\n전주투어 릴레이 중\n여의도 불꽃축제 리뷰 쓰고 돌아...
16004,https://blog.naver.com/thevioletrabbit/2228980...,주간일기챌린지 0925-1001,2022.10.12.,9월 25일 일요일\nNext image\n런데이 처음 한 날! 오랜만에 뛰니까 배...
...,...,...,...,...
16995,https://blog.naver.com/minekooo/222896824517,2022 ACC 사진공모전 국립아시아문화전당과 함께,2022.10.10.,#2022ACC사진공모전 #국립아시아문화전당\n국립아시아문화전당\n2022 ACC ...
16996,https://blog.naver.com/rush98y/222896821509,종로구 흥인지문공원 낙산성곽길 이화마을,2022.10.10.,종로구 흥인지문공원을 둘러보고 성곽길을 따라 이화벽화마을을 구경했어요~\n낙산공원까...
16997,https://blog.naver.com/doubt10454/222896821614,대구성서 수학과외 중등 고등학생 이곡동 영어과외 국어 과학,2022.10.10.,대구성서 수학과외 중등 고등학생 이곡동 영어과외 국어 과학\n초등학생의 기초학력 수...
16998,https://blog.naver.com/gksrkdms1115/222896820233,"[태국/방콕여행] 1일차 - 인천공항 마티나라운지, 수완나품공항...",2022.10.10.,안녕하세요~~\n이번에는 방콕여행 1일차\n후기를 가지고 돌아왔습니다 ㅎ ㅎ\n블로...


In [38]:
df17.to_excel("D:/2022산림문화/2022나무문화/본문/2022나무문화_62차 본문.xlsx", index=False, encoding='UTF-8')
df17.to_csv("D:/2022산림문화/2022나무문화/본문/2022나무문화_62차 본문.txt", index=False, encoding='UTF-8')


C:\Users\user\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'https://www.youtube.com/watch?v=muW3NCNOnR0
발길%20가는%20대로%20걷자,%20켜켜이%20쌓여온%20마음%20속%20응어리가%20쑥%20내려갔다.
교사%20은퇴%20후%20길%20위의%20자유를%20만끽하는%2070대%20도보%20여행가%20이야기｜%5b국내여행%5d%20KBS%2020170111
%5b%20사람과사람들%2062회%20-여보,%20또%20어디가?%20%5d
평생%20교직에%20몸담아오다%20예순다섯에%20도보여행을%20시작한%20황경화%20할머니.
10여년이%20넘는%20시간%20동안%20지구%20두%20바퀴%20반%20가까운%20거리를%20걸어온%20할머니에게%20길은%20무엇이었을까.
https://bravo.etoday.co.kr/view/atc_view.php?varAtcId=4857
%5b세상%20참%20걸을%20만하구나%20PART1%5d%20소박한%20일탈,%20걷기부터
▲사진%20신석교%20프리랜서%20사진가%20건강전문가들은%20건강을%20위해%20일생동안%20꾸준히%20할%20수%20있는%20운동은%20언제%20어디서나%20쉽게%20할%20수%20있으며%20효과가%20큰%20것이어야
bravo.etoday.co.kr
%5b세상%20참%20걸을%20만하구나%20PART1%5d%20소박한%20일탈,%20걷기부터
기사입력%202016-04-19%2009:29기사수정%202016-04-25%2017:27
▲사진%20신석교%20프리랜서%20사진가
건강전문가들은%20건강을%20위해%20일생동안%20꾸준히%20할%20수%20있는%20운동은%20언제%20어디서나%20쉽게%20할%20수%20있으며%20효과가%20큰%20것이어야%20한다고%20말한다.%20그래서%20요즘%20‘걷기’가%20유행이다.%20걷기는%20조기%20사망%20위

In [39]:
hide_link


[]